> Apr 12, 2024 | {sub-ref}`wordcount-minutes` min read


$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\i}{{\color{blue} i}} $ 
$ \newcommand{\Hil}{{\cal H}} $
$ \newcommand{\cg}[1]{{\rm C}#1} $
$ \newcommand{\lp}{\left(} $
$ \newcommand{\rp}{\right)} $
$ \newcommand{\lc}{\left[} $
$ \newcommand{\rc}{\right]} $
$ \newcommand{\lch}{\left\{} $
$ \newcommand{\rch}{\right\}} $
$ \newcommand{\Lp}{\Bigl(} $
$ \newcommand{\Rp}{\Bigr)} $
$ \newcommand{\Lc}{\Bigl[} $
$ \newcommand{\Rc}{\Bigr]} $
$ \newcommand{\Lch}{\Bigl\{} $
$ \newcommand{\Rch}{\Bigr\}} $
$ \newcommand{\rqa}{\quad \Rightarrow \quad} $

(sec_FTA_grover_cons_imple)= 
# 6. Consideraciones sobre la implementación

:::{contents}
:local:
:depth: 1
:::


(sec_FTA_grover_sub_U_s)= 
## Creación de un difusor $U_{\Psi_0}$.

Vamos a ver como podemos hacer para crear de forma genérica un difusor de la forma de la Ec. {eq}`ec_FTA_grover_t-sols_Uw_Us`. Refrescando un poco la memoria, el difusor es un operador realiza una reflexión respecto al estado inicial $| \Psi_0 \rangle$, es decir, **le cambia el signo a las componentes perpendiculares a** $| \Psi_0 \rangle$. Lo que vamos a hacer para construir el difusor es, en realidad, construir un operador que  **le cambia el signo a las componentes paralelas a** $| \Psi_0 \rangle$, es decir, vamos a implementar $-U_{\Psi_0}$. 

Empecemos definiendo la familia de **operadores de reflexión** $S_A$


```{math}
:label: ec_FTA_grover_consideraciones_S_a 
\begin{equation} 
\boxed{S_A | i \rangle = 
	\lch
		\begin{matrix}
			& | i \rangle \quad \text{si } i \not \in A \\
			& - | i \rangle \quad \text{si } i \in A
		\end{matrix}
	\right.}
\end{equation} 
```

Es fácil ver que podemos escribir tanto el oráculo como el difusor en función de los operadores $S_A$

\begin{align}
& U_\omega = S_\omega
& \boxed{U_{\Psi_0} = - S_{\Psi_0}}
\end{align}

Nosotros lo que vamos a construir y implementar es $S_{\Psi_0}$, no $U_{\Psi_0}$

(sec_FTA_grover_sub_caso_N_eq_2n)= 
### Caso con $N = 2^n$.

En el caso en el que el estado inicial es una superposición uniforme de la forma la Ec. {eq}`ec_FTA_grover_geo_phi_0_H` podemos construir el difusor teniendo en cuenta que

\begin{equation}
|\Psi_0 \rangle = H^{\otimes n} |0 \rangle \rqa H^{\otimes n} |\Psi_0 \rangle = |0 \rangle.
\end{equation}

Viendo esta propiedad, podemos darnos cuenta de que si aplicamos el operador de Walsh-Hadamard $H^{\otimes n}$ a la salida del oráculo lo que obtenemos es el estado $| 0 \rangle$ más una serie de estado que corresponderán a los cambios respecto al estado inicial que ha realizado el oráculo. Lo que tenemos que hacer para aplicar el difusor es cambiarle el signo al estado $| 0 \rangle$ (aplicar $S_0$) y volver a aplicar $H^{\otimes n}$ para deshacer los cambios introducidos por la última aplicación el mismo.

Es decir, el difusor será de la forma

\begin{equation}
U_{\Psi_0} = - S_{\Psi_0}= - H^{\otimes n} S_0 H^{\otimes n}
\end{equation}

Podemos construir $S_0$ a partir de la **puerta multicontrolada Z** ($MCZ$)

\begin{equation}
MCZ = 
	\lp
		\begin{matrix}
			& 1 & 0  & \dots & 0 & 0 \\
			& 0 & 1  & \dots & 0 & 0 \\
			& \vdots  & \vdots & \ddots & \vdots & \vdots \\
			& 0 & 0 & \dots & 1 & 0 \\
			& 0 & 0 & \dots & 0 & -1 \\
		\end{matrix}
	\rp
\end{equation}

que lo que hace es cambiarle el signo al estado $|2^n-1\rangle= | 11 \dots 1\rangle$.

Tenemos pues

\begin{equation}
S_0 = X^{\otimes n} (MCZ) X^{\otimes n}
\end{equation}

La puerta $X^{\otimes n}$ (que consiste en aplicar puertas $X$ a todos los qubits) lo que hace es 

\begin{align*}
& |00 \dots 0 \rangle \rightarrow |11 \dots 1 \rangle \, \, \Lc |0 \rangle \rightarrow | 2^n-1 \rangle \Rc \\
& |11 \dots 1 \rangle \rightarrow |00 \dots 0 \rangle \, \, \Lc |2^n-1 \rangle \rightarrow | 0 \rangle \Rc
\end{align*}

De esta forma, lo que hace $S_{0}$ es: 

- Aplicar la puerta $X^{\otimes n}$ para cambiar el estado $|00 \dots 0 \rangle$ por el estado $|11 \dots 1 \rangle$. (Ver la Nota de abajo)
	
- Aplicar la puerta $MCZ$ con la que cambiamos el signo a $|11 \dots 1 \rangle$ 
	
- Aplicar la puerta $X^{\otimes n}$ para deshacer los cambios del primer paso. De esta forma, el único cambio real es el del signo del estado $|0\rangle = |00 \dots 0 \rangle$.

::::::{admonition} Nota
:class: note


     
En realidad la puerta $X^{\otimes n}$ afecta a todos los estados (no solo a $|00 \dots 0 \rangle $ y $|11 \dots 1 \rangle$).
Sin embargo, como es su propia inversa ($X^{\otimes n}X^{\otimes n} = I$) y como entre la primera y la segunda 	aplicación de $X^{\otimes n}$ lo único que hacemos 	es cambiarle al signo a $|11 \dots 1 \rangle$, todos los 	cambios se deshacen menos este signo, que pasa a estar en el estado $|00 \dots 0 \rangle $. 
::::::




Como ya comentamos, **el operador que se implementa es** $-U_{\Psi_0}$, es decir


```{math}
:label: ec_FTA_grover_cons-imple_U_Psi0 
\begin{align} 
& \boxed{-U_{\Psi_0}  = S_{\Psi_0}  = H^{\otimes n} S_0 H^{\otimes n} = H^{\otimes n} X^{\otimes n} (MCZ) X^{\otimes n} H^{\otimes n}} \Rightarrow  \\ 
& \boxed{G_{imple}  =  - U_{\Psi_0} U_\omega = - H^{\otimes n} S_0 H^{\otimes n} U_\omega}
\end{align} 
```

(sec_FTA_grover_sub_N-no-2^n)= 
(sec_FTA_grover_subsec-N_neq_2n)= 
### Caso con $N \neq 2^n$.

Las limitación que hemos impuesto hasta ahora diciendo que $N$ debía ser una potencia de $2$ viene de la transformación de Walsh-Hadammard 

\begin{equation}
H^{\otimes n} |j \rangle = \frac{1}{\sqrt{N}} \sum^{N-1}_{i=0} (-1)^{i \cdot j} | i \rangle, \qquad \text{ (donde } i \cdot j \text{ denota el producto escalar binario)}
\end{equation}

Esta transformación, que se usa para generar el estado inicial y se en el difusor, no está bien definida si no se cumple que $N=2^n$.

Esta condición puede ser relajada si **sustituimos la transformación de Walsh-Hadammard por cualquier otra transformación unitario $F$ que cumpla**


```{math}
:label: ec_FTA_grover_N-no-2n_T 
\begin{equation} 
\boxed{F |0 \rangle = \frac{1}{\sqrt{N}} \sum^{N-1}_{i=0} | i \rangle}
\end{equation} 
```

y seguiremos teniendo interaciones de Grover válidas aplicando 

\begin{equation}
F S_0 F^{-1} U_\omega
\end{equation}

De hecho, cuando estamos en el caso $N = 2^n$, la Walsh-Hadammard es simplemente la elección más sencilla de $F$. Para el caso en el que $N$ no es una potencia de dos, podemos usar la transformación de Fourier de Kitaev {cite}`bib_FTA_Grover_kitaev1995quantum`. Puede verse también {cite}`bib_FTA_Grover_PhysRevLett_80_4329`

---
## Bibliografía 
```{bibliography} 
:style: plain
:filter: docname in docnames
```

---
Autores:

David Castaño ([UMA-SCBI](https://www.scbi.uma.es/site/)), Raul Fuentes ([BSC-CNS](https://www.bsc.es/es)), Daniel Talaván  ([COMPUTAEX](https://www.cenits.es/)), Francisco Matanza ([UNICAN](https://web.unican.es/))

<img src="images/Logo_UMA.jpeg"  width="170px">
<img src="images/BSC-blue-medium.png" width="195px">
<img src="images/COMPUTAEX.jpg" width="170px">
<img src="images/Logo_UNICAM.jpg" width="210px">


---
::::{figure} https://quantumspain-project.es/wp-content/uploads/2022/11/Logo_QS_EspanaDigital.png
:width: 2000px
:align: center
::::

<center>
<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">
<img aling="left" alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a>
</center>

License: <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Licencia Creative Commons Atribución-CompartirIgual 4.0 Internacional</a>.

This work has been financially supported by the Ministry for Digital Transformation and of Civil Service of the Spanish Government through the QUANTUM ENIA project call - Quantum Spain project, and by the European Union through the Recovery, Transformation and Resilience Plan - NextGenerationEU within the framework of the Digital Spain 2026 Agenda.

<center>
<img align="left" src="https://quantumspain-project.es/wp-content/uploads/2024/02/Banner-QS_GOB_v2.png" width="1000px" />
</center>